#第8章
第6章で取り組んだニュース記事のカテゴリ分類を題材として，ニューラルネットワークでカテゴリ分類モデルを実装する．なお，この章ではPyTorch, TensorFlow, Chainerなどの機械学習プラットフォームを活用せよ．

**PyTorch** を使う

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/nlp100/'

#70. 単語ベクトルの和による特徴量
問題50で構築した学習データ，検証データ，評価データを行列・ベクトルに変換したい．例えば，学習データについて，すべての事例xi
の特徴ベクトルxi
を並べた行列X
と，正解ラベルを並べた行列（ベクトル）Y
を作成したい．

仕様に基づき，以下の行列・ベクトルを作成し，ファイルに保存せよ．

* 学習データの特徴量行列: Xtrain
* 学習データのラベルベクトル: Ytrain
* 検証データの特徴量行列: Xvalid
* 検証データのラベルベクトル: Yvalid
* 評価データの特徴量行列: Xtest
* 評価データのラベルベクトル: Ytest

In [ ]:
!unzip /content/drive/MyDrive/nlp100/NewsAggregatorDataset.zip

Archive:  /content/drive/MyDrive/nlp100/NewsAggregatorDataset.zip
replace 2pageSessions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 2pageSessions.csv       
replace __MACOSX/._2pageSessions.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/._2pageSessions.csv  
replace newsCorpora.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: newsCorpora.csv         y

replace __MACOSX/._newsCorpora.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: __MACOSX/._newsCorpora.csv  
replace readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: readme.txt              
replace __MACOSX/._readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/._readme.txt   


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk

nltk.download('punkt')
data_path = '/content/newsCorpora.csv'

publisheres = ["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"]
names = ["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"]
csv_input = pd.read_csv(data_path, sep="\t", header=None, names=names)

df = csv_input[csv_input["PUBLISHER"].isin(publisheres)]

df = df.sample(frac=1)

# 訓練：検証：評価=8:1:1に分割
x_train, x_valtest = train_test_split(df.values, test_size=0.2, random_state=0)
x_valid, x_test = train_test_split(x_valtest, test_size=0.5, random_state=0)
# option:strtify  ラベルの偏りをなくす

# FORMAT: ID \t TITLE \t URL \t PUBLISHER \t CATEGORY \t STORY \t HOSTNAME \t TIMESTAMP
def write_file(fname, x_data):
    with open(fname, "w")as fout:
        for fact in x_data:
            # fact[1] : 見出し
            # fact[4] : カテゴリ
            # b = business, t = science and technology, e = entertainment, m = health
            if fact[4] == 'b':
                label = '0'
            elif fact[4] == 't':
                label = '1'
            elif fact[4] == 'e':
                label = '2'
            elif fact[4] == 'm':
                label = '3'
            text = " ".join(nltk.word_tokenize(fact[1]))
            fout.write(label + '\t' + text + '\n')
            
write_file("train.txt", x_train)
write_file("valid.txt", x_valid)
write_file("test.txt", x_test)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# !gzip -d /content/drive/MyDrive/nlp100/GoogleNews-vectors-negative300.bin.gz

gzip: /content/drive/MyDrive/nlp100/GoogleNews-vectors-negative300.bin.gz: No such file or directory


In [ ]:
import gensim

model_path = '/content/drive/MyDrive/nlp100/GoogleNews-vectors-negative300.bin'
w2v = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [ ]:
# import numpy as np
# # 学習データの特徴量行列: Xtrain
# # 学習データのラベルベクトル: Ytrain
# # 検証データの特徴量行列: Xvalid
# # 検証データのラベルベクトル: Yvalid
# # 評価データの特徴量行列: Xtest
# # 評価データのラベルベクトル: Ytest

def save_file(infname):
    with open(infname, "r") as fin:
        with open("X" + infname, "w") as fvec, open("Y" + infname, "w") as flabel:
            for line in fin:
                label, text = line.strip().split('\t')
                try:
                    mean = np.mean([w2v[word] for word in text.split(' ')], axis=1)
                    fvec.write(str(mean) + '\n')
                    flabel.write(label + '\n')
                except:
                    continue

save_file("train.txt")
save_file("valid.txt")
save_file("test.txt")

In [ ]:
import numpy as np
import torch
# 学習データの特徴量行列: Xtrain
# 学習データのラベルベクトル: Ytrain
# 検証データの特徴量行列: Xvalid
# 検証データのラベルベクトル: Yvalid
# 評価データの特徴量行列: Xtest
# 評価データのラベルベクトル: Ytest

def create_data(infname):
    with open(infname, "r") as fin:
        vec = []
        y_label = []
        for line in fin:
            label, text = line.strip().split('\t')
            try:
                # mean = np.mean([w2v[word] for word in text.split(' ')], axis=1)
                tmp_vec = [w2v[word] for word in text.split(' ')]
                mean = sum(tmp_vec) / len(tmp_vec)

                # print(mean)
                vec.append(mean)
                y_label.append(int(label))
            except:
                continue
        return vec, y_label

x_train, y_train = create_data("train.txt")
x_valid, y_valid = create_data("valid.txt")
x_test, y_test = create_data("test.txt")

In [ ]:
x_train[1]

array([-0.02466901,  0.03905233, -0.03706868,  0.11861674,  0.00026449,
        0.02161916, -0.05620956, -0.03760274,  0.12687175,  0.18186443,
       -0.05770747, -0.02668254, -0.11054484,  0.06424967, -0.02377701,
       -0.11365763,  0.04849752,  0.05368741, -0.04326884,  0.09051514,
        0.0265096 ,  0.01118978,  0.09823608,  0.03806559, -0.02258809,
        0.03462728, -0.04766146, -0.00567627, -0.03023275,  0.07553101,
       -0.06899516, -0.07203484, -0.00072734, -0.19705708, -0.0338796 ,
       -0.05609131,  0.02432887,  0.03576152, -0.04305013,  0.09376017,
        0.0107015 , -0.17664592, -0.04592196, -0.08171082,  0.05003865,
       -0.00632032, -0.11639658, -0.04033406,  0.06881714,  0.0835228 ,
       -0.2273051 ,  0.15090434,  0.00738525,  0.00463359,  0.06213124,
        0.03665606, -0.17370605, -0.00270844, -0.0353597 , -0.00565592,
       -0.09235636,  0.08309937, -0.1047109 , -0.11782201, -0.10122681,
       -0.11846288,  0.02043661,  0.06773885,  0.11298624,  0.00

In [ ]:
#ファイルへの出力
import pickle

# ラベルもtensorに変換
train_t = torch.tensor(y_train).long()
valid_t = torch.tensor(y_valid).long()
test_t = torch.tensor(y_test).long()

with open('train.text.pickle', 'wb') as f:
    pickle.dump(x_train, f)
with open('train.label.pickle', 'wb') as f:
    pickle.dump(y_train, f)

with open('valid.text.pickle', 'wb') as f:
    pickle.dump(x_valid, f)
with open('valid.label.pickle', 'wb') as f:
    pickle.dump(y_valid, f)

with open('test.text.pickle', 'wb') as f:
    pickle.dump(x_test, f)
with open('test.label.pickle', 'wb') as f:
    pickle.dump(y_test, f)


#71. 単層ニューラルネットワークによる予測
問題70で保存した行列を読み込み，学習データについて以下の計算を実行せよ．

In [ ]:
#ネットワークの定義
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self, input_size, output_size):
    super().__init__()
    self.fc = nn.Linear(input_size, output_size, bias=False)
    nn.init.normal_(self.fc.weight, 0.0, 1.0)  # 正規乱数で重みを初期化

  def forward(self, x):
    x = self.fc(x)
    return x

# ネットワークのインスタンスを作成
net = Net(300, 4)
net

Net(
  (fc): Linear(in_features=300, out_features=4, bias=False)
)

In [ ]:
x_train = torch.from_numpy(np.array(x_train))  
y_train = torch.from_numpy(np.array(y_train))  

In [ ]:
x = net(x_train[0])
x = torch.softmax(x, dim=-1)
x

tensor([0.2545, 0.5996, 0.1091, 0.0368], grad_fn=<SoftmaxBackward0>)

In [ ]:
x = net(x_train[:4])
x = torch.softmax(x, dim=-1)
x

tensor([[0.2545, 0.5996, 0.1091, 0.0368],
        [0.3614, 0.0487, 0.0135, 0.5765],
        [0.6587, 0.2773, 0.0180, 0.0461],
        [0.3439, 0.2655, 0.1719, 0.2187]], grad_fn=<SoftmaxBackward0>)

In [ ]:
# 乱数の初期化：いろんな初期化がある．初期値で性能が変わることもある．

#72. 損失と勾配の計算
学習データの事例x1
と事例集合x1,x2,x3,x4
に対して，クロスエントロピー損失と，行列W
に対する勾配を計算せよ．なお，ある事例xi
に対して損失は次式で計算される．

li=−log[事例xiがyiに分類される確率]
ただし，事例集合に対するクロスエントロピー損失は，その集合に含まれる各事例の損失の平均とする．

In [ ]:
import torch.nn as nn

# 損失の計算
criterion = nn.CrossEntropyLoss() # クロスエントロピーの中でsoftmaxの計算を計算を行っている．softmaxをかける前に入力する．
y = net(x_train[:1])
t = y_train[:1]
loss = criterion(y, t)

# 勾配の計算
net.zero_grad()
loss.backward()

print('損失 :', loss.item())
print('勾配 :')
print(net.fc.weight.grad)

損失 : 2.8086366653442383
勾配 :
tensor([[-0.0059,  0.0162, -0.0017,  ..., -0.0212,  0.0188, -0.0310],
        [-0.0030,  0.0081, -0.0008,  ..., -0.0106,  0.0094, -0.0155],
        [ 0.0154, -0.0422,  0.0044,  ...,  0.0552, -0.0490,  0.0808],
        [-0.0065,  0.0179, -0.0019,  ..., -0.0234,  0.0208, -0.0343]])


In [ ]:
# 損失の計算
criterion = nn.CrossEntropyLoss()
y = net(x_train[:4])
t = y_train[:4]
loss = criterion(y, t)

# 勾配の計算
net.zero_grad()
loss.backward()

print('損失 :', loss.item())
print('勾配 :')
print(net.fc.weight.grad)

損失 : 2.076000928878784
勾配 :
tensor([[ 0.0028, -0.0098,  0.0440,  ...,  0.0047, -0.0193,  0.0271],
        [-0.0005,  0.0031, -0.0018,  ..., -0.0035,  0.0037, -0.0052],
        [ 0.0077, -0.0010, -0.0213,  ..., -0.0013,  0.0068,  0.0029],
        [-0.0099,  0.0077, -0.0210,  ...,  0.0001,  0.0089, -0.0249]])


#73. 確率的勾配降下法による学習
確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，行列W
を学習せよ．なお，学習は適当な基準で終了させればよい（例えば「100エポックで終了」など）．

In [ ]:
x_valid = torch.from_numpy(np.array(x_valid))  
y_valid = torch.from_numpy(np.array(y_valid)) 
x_test = torch.from_numpy(np.array(x_test))  
y_test = torch.from_numpy(np.array(y_test))  

In [ ]:
# ネットワークのインスタンスを作成
net = Net(300, 4)

#損失関数の選択
criterion = nn.CrossEntropyLoss()

#最適化手法の選択
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

# モデルをGPUに転送
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = net.to(device)
device

device(type='cpu')

In [ ]:
#DataLoaderに格納

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

train = TensorDataset(x_train, y_train)
val = TensorDataset(x_valid, y_valid)
test = TensorDataset(x_test, y_test)

batch_size = 10

train_loader = DataLoader(train, batch_size, shuffle=True)
val_loader = DataLoader(val, batch_size, shuffle=False)
test_loader = DataLoader(test, batch_size, shuffle=False)
eval_loader = DataLoader(eval, batch_size, shuffle=False)

In [ ]:
#学習ループ 
max_epoch = 20

for epoch in range(max_epoch):

    # ミニバッチ学習
    for batch in train_loader:

        # バッチサイズ分のサンプルを抽出
        x, t = batch  

        # データをGPUへ転送
        x = x.to(device)
        t = t.to(device)

        # 勾配を初期化
        optimizer.zero_grad()

        # 順伝播
        y = net(x) 
        loss = criterion(y, t) 

        # 誤差逆伝播
        loss.backward()  
        optimizer.step() 
    
    # 更新と切り離し、検証データの性能を確認
    with torch.no_grad():
        losses = list()
        for batch in val_loader:
            x, t = batch 
            x = x.to(device)
            t = t.to(device)
            y = net(x)  
            loss = criterion(y, t) 
            losses.append(loss)
    val_loss = torch.tensor(losses).mean()
    print("Epoch: %02d  val_loss: %.3f" % (epoch+1, val_loss))

Epoch: 01  val_loss: 0.818
Epoch: 02  val_loss: 0.672
Epoch: 03  val_loss: 0.636
Epoch: 04  val_loss: 0.606
Epoch: 05  val_loss: 0.599
Epoch: 06  val_loss: 0.590
Epoch: 07  val_loss: 0.589
Epoch: 08  val_loss: 0.590
Epoch: 09  val_loss: 0.599
Epoch: 10  val_loss: 0.601
Epoch: 11  val_loss: 0.596
Epoch: 12  val_loss: 0.599
Epoch: 13  val_loss: 0.605
Epoch: 14  val_loss: 0.608
Epoch: 15  val_loss: 0.617
Epoch: 16  val_loss: 0.620
Epoch: 17  val_loss: 0.623
Epoch: 18  val_loss: 0.625
Epoch: 19  val_loss: 0.632
Epoch: 20  val_loss: 0.637


#74. 正解率の計測
問題73で求めた行列を用いて学習データおよび評価データの事例を分類したとき，その正解率をそれぞれ求めよ．

In [ ]:
from sklearn.metrics import accuracy_score

net.eval()
with torch.no_grad():
    losses = list()
    total = 0
    correct = 0

    for batch in val_loader:
        x, t = batch

        x = x.to(device)
        t = t.to(device)
        y = net(x)
        pred = torch.argmax(y, dim=-1)

        total += len(x)
        correct += (pred == t).sum().item()

print("valid 正解率: ", correct / total)

valid 正解率:  0.8012422360248447


In [ ]:
net.eval()
with torch.no_grad():
    losses = list()
    total = 0
    correct = 0

    for batch in test_loader:
        x, t = batch

        x = x.to(device)
        t = t.to(device)
        y = net(x)
        pred = torch.argmax(y, dim=-1)

        total += len(x)
        correct += (pred == t).sum().item()

print("test 正解率: ", correct / total)

test 正解率:  0.8759124087591241


#75. 損失と正解率のプロット
問題73のコードを改変し，各エポックのパラメータ更新が完了するたびに，訓練データでの損失，正解率，検証データでの損失，正解率をグラフにプロットし，学習の進捗状況を確認できるようにせよ．

#76. チェックポイント
問題75のコードを改変し，各エポックのパラメータ更新が完了するたびに，チェックポイント（学習途中のパラメータ（重み行列など）の値や最適化アルゴリズムの内部状態）をファイルに書き出せ．

#77. ミニバッチ化
問題76のコードを改変し，B
事例ごとに損失・勾配を計算し，行列W
の値を更新せよ（ミニバッチ化）．B
の値を1,2,4,8,…
と変化させながら，1エポックの学習に要する時間を比較せよ．

#78. GPU上での学習
問題77のコードを改変し，GPU上で学習を実行せよ．

#79. 多層ニューラルネットワーク
問題78のコードを改変し，バイアス項の導入や多層化など，ニューラルネットワークの形状を変更しながら，高性能なカテゴリ分類器を構築せよ．